# AI2-Cat Calculation Notebook
此 Notebook 将演示如何从一个结构文件出发，借助 `ai2-cat` 所提供的工具构建执行 AIMD, 势函数训练和性质计算的配置文件并执行以得到所需求数据的方法。

1. 提供初始结构，进行AIMD模拟获取获取轨迹文件与相关能量和力等信息 (可选)
2. 收集轨迹中的结构与对应的能量和力转化为机器学习势函数可训练的数据集格式
3. 基于上述的训练数据集训练机器学习势函数
4. 基于上述的机器学习势函数进行分子动力学模拟+增强采样，获取新的结构
5. 基于力误差判据筛选出新的结构
6. 对上述的新结构进行第一性原理计算获取结构对应的能量和力并加入数据集
7. 迭代步骤3-6直至步骤5无法筛选出新的构象
8. 基于最后的机器学习势函数进行分子动力学模拟+增强采样
9. 基于模拟结果计算反应自由能


In [ ]:
%matplotlib inline

from ai2_kit.feat import catalysis as ai2cat
import os
AI4EC_DIR = '/public/groups/ai4ec/data'  # 公共资源目录
CP2K_DATA = f'{AI4EC_DIR}/cp2k'
CP2K_BATCH_TEMPLATE = f'{AI4EC_DIR}/slurm/cp2k.sbatch'
ui = ai2cat.get_the_ui_helper()

#dev
CP2K_DATA = "./"
AI4EC_DIR = "./"

# 初始数据集准备
初始数据集是启动势函数训练的必要输入。如果已经有适用于 DeepMD 的初始数据集和用于 LAMMPS 结构搜索的初始结构，则可以直接上传相应数据直接进入下一步的势函数训练中。

如果没有所需的数据，则可以采用本节所提供的 AIMD 相关功能生成所需的数据。

## 生成 AIMD 配置
计算所需要的结构文件（支持 xyz, cif, POSCAR 等格式) 需自行构建并上传，然后使用以下命令即可生成用于 AIMD 执行的 CP2K 配置。

In [ ]:
! mkdir -p ./00-aimd
ui.gen_aimd_config(cp2k_search_path=CP2K_DATA, out_dir='./00-aimd')

此时 `00-aimd` 目录里能找到生成的 CP2K 配置和数据文件。
通过左边栏打开文件可进行检查或编辑。

确认无误后可使用 CP2K 作业模板提交任务。

## 提交 AIMD 作业

In [ ]:
# 复制作业提交脚本到任务目录 (复制后可根据需要进行调整)
! cp {CP2K_BATCH_TEMPLATE} ./00-aimd/

In [ ]:
# 提交作业
! (cd ./00-aimd && sbatch cp2k.sbatch)

In [ ]:
# 检查作业执行情况
! squeue -u $USER

### 生成 DeepMD 数据集和初始文件

AIMD 作业完成后会在工作目录下生成相应的结构和力数据。通过 `ai2-kit` 所提供的数据格式处理工具可以很轻松地完成这项任务。

相关工具的详细使用文档，可参考 [ai2-kit ase](https://github.com/chenggroup/ai2-kit/blob/main/doc/manual/ase.md) 和 [ai2-kit dpdata](https://github.com/chenggroup/ai2-kit/blob/main/doc/manual/dpdata.md)

In [ ]:
# 生成初始训练数据集
! ai2-kit tool dpdata read ./00-aimd --fmt cp2kdata/md --cp2k_output_name=output - write ./data/train

In [ ]:
# 生成初始结构
# 以下命令以每 20 帧为间隔从 AIMD 轨迹中抽取结构
# 并限制最大帧数为 50
# 使用 `set_by_ref` 从结构文件获取 cell, pbc 等信息
# 然后以 xyz 格式写入指定位置
# 实际使用时可根据实际需求对参数进行调整
! ai2-kit tool ase read ./00-aimd/*-pos-1.xyz --index '::20' - limit 50 - set_by_ref POSCAR - write ./data/explore/init.xyz

# 训练势函数

## 生成 ai2-kit 势函数训练工作流配置

AIMD 作业顺利完成后可从其输出得到轨迹数据。这里可同样借助 `ai2-cat` 自动生成相关配置文件。需要生成的文件包括
1. CP2K 配置文件 cp2k.inp
2. DeepMD 配置文件 deepmd.json
3. PLUMED 配置文件 plumed.dat
4. ai2-kit cll-mpl-training 配置文件 training.yml, executor.yml

In [ ]:
! ai2-cat build_config load_system POSCAR - gen_deepmd_input --out_dir ./01-train
! ai2-cat build_config load_system POSCAR - gen_cp2k_input \
  --basic_set_file /public/groups/ai4ec/data/cp2k/BASIS_MOLOPT \
  --potential_file /public/groups/ai4ec/data/cp2k/GTH_POTENTIALS \
  --out_dir ./01-train
! ai2-cat build_config load_system POSCAR - gen_plumed_input --out_dir ./01-train
! ai2-cat build_config load_system POSCAR - gen_mlp_training_input --out_dir ./01-train
! cp /public/groups/ai4ec/data/ai2-kit/executor.yml ./01-train

## 编辑 `ai2-kit` 势函数训练工作流配置
目前由于技术方案的限制，自动生成的配置文件中仍有部分需要人工进行调整，其最小改动如下
* `plumed.dat`, plumed 的配置, 需要由用户定义增强采样的反应坐标, 以及其它所需的命令
  * 例如增加 `CV1: DISTANCE ATOMS=14,15`，根据具体的研究目标进行配置
* `training.yml`，需要用户指定使用的数据路径，并在工作流中引用
  * 例如在 `artifacts` 一节中需要配置所需使用的体系等

更具体的配置文档可参考以下 [文档](https://github.com/chenggroup/ai2-kit/blob/main/doc/manual/cll-workflow.zh.md) 和 [配置示例](https://github.com/chenggroup/ai2-kit/tree/main/example/config/cll-mlp-training)。

## 执行 `ai2-kit` 势函数训练工作流

配置确认无误后，即可使用以下命令在后台启动训练。

In [ ]:
project_prefix = 'ai2cat-demo/run-01'  # 请根据项目的工作目录修改
os.system(f"cd ./01-train && nohup ai2-kit workflow cll-mlp-training *.yml --executor ikkem --path-prefix {project_prefix} &")

In [ ]:
# 通过以下命令查看日志输出以确定执行是否正常
! tail -f ./01-train/nohup.out

## 阶段性结果检查
在很多情况下，尤其是针对复杂体系，势函数的训练往往不是一蹴而就。因此，一种比较稳健的策略阶段性地执行工作流，即在训练开始时指定较小的 `max_iter`, 在训练完成后对输出进行检查，并根据结果对配置进行必要的调整 (手动打开配置文件进行更新) 再次执行相同的训练命令，此时工作流会继承先前的状态，继续执行后续的迭代，直到势函数的训练结果满足要求。

以下使用 `ai2-cat` 工具包中提供的工具对结果进行检查。

### 检查 DeepMD lcurve.out 文件

In [ ]:
from ai2_kit.tool.deepmd import display_lcurve
lcurve_file = '/path/to/lcurve.out'
display_lcurve(lcurve_file)

### 检查 LAMMPS 输出

In [ ]:
lammps_task_dir = '/path/to/lammps/task'
ai2cat.inspect_explore_result(lammps_dir)

# 基于势函数进行动态催化模拟

在得到满足要求的势函数后，即可使用势函数进行动态催化模拟以得到相关性质，并对输出结果进行分析得到报告。

## 使用势函数生产性质所需数据

## 生成报告